# 資本予算

In [1]:
import pandas as pd

In [2]:
data = pd.DataFrame(data=[
    [1, 100, 200],
    [2, 300, 500],
    [3, 200, 300],
    [4, 150, 200],
    [5, 150, 250],
    ],
    columns=['プロジェクト', '支出', '現在価値']).set_index('プロジェクト')
data

,支出,現在価値
プロジェクト,,
1,100,200
2,300,500
3,200,300
4,150,200
5,150,250


利益・費用率の計算

In [3]:
data['利益・費用率'] = data['現在価値']/data['支出']
data

,支出,現在価値,利益・費用率
プロジェクト,,,
1,100,200,2.000000
2,300,500,1.666667
3,200,300,1.500000
4,150,200,1.333333
5,150,250,1.666667


利益・費用率で並び替え(sort)

In [4]:
sort_data = data.sort_values('利益・費用率', ascending=False)
sort_data

,支出,現在価値,利益・費用率
プロジェクト,,,
1,100,200,2.000000
2,300,500,1.666667
5,150,250,1.666667
3,200,300,1.500000
4,150,200,1.333333


## 近似法の場合
利益・費用率の高いプロジェクトから順に、支出の合計が600を超えるまで採択する。

In [5]:
project = sort_data[sort_data['支出'].cumsum()<600]
project.loc['合計'] = project.sum(axis=0)
project

,支出,現在価値,利益・費用率
プロジェクト,,,
1,100.0,200.0,2.000000
2,300.0,500.0,1.666667
5,150.0,250.0,1.666667
合計,550.0,950.0,5.333333


よって近似法の場合はプロジェクト1, 2, 5 が採択される。

## 最適集合の場合
$i$番目のプロジェクトの採択の有無をバイナリ変数$x_i$（採択：1 不採択：0）、現在価値を$b_i$、支出を$c_i$とする。<br>
最適化の定式化は、
\begin{align}
\max_{\bf x}&\sum_{i=1}^mb_ix_i\\
s.t. &\sum_{i=1}^mc_ix_i\leq C
\end{align}

In [18]:
from pulp import *
m = LpProblem(sense=LpMaximize) # 最大化問題として定式化
x = [LpVariable('x{0}'.format(i+1), cat=LpBinary) for i in range(len(data))] # 0-1の変数xの設定
m += (data['現在価値']*x).sum() # 目的関数
m += (data['支出']*x).sum() <= 600 # 制約条件

In [19]:
m.solve() # ソルバーの実行

1

In [36]:
ans = [bool(value(x[i])) for i in range(len(data))]
project = data[ans]
project.loc['合計'] = project.sum(axis=0)
project

,支出,現在価値,利益・費用率
プロジェクト,,,
1,100.0,200.0,2.000000
2,300.0,500.0,1.666667
3,200.0,300.0,1.500000
合計,600.0,1000.0,5.166667


よって最適解の場合はプロジェクト1, 2, 3 が採択される。